In [54]:
import ipy_table
from collections import Counter
from IPython.display import HTML, display
import tabulate
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
import warnings

warnings.filterwarnings('ignore')
%run ../aux/postgres.py

postgres functions loaded...


In [14]:
def matrixConfusao(av01, av02, labels, labelsN):
    a = confusion_matrix(av01, av02, labels=labels).tolist()
    a.insert(0,labelsN)
    labelsNames = [''] + labelsN
    b = []
    i = 0
    for x in a:
        x = list(x)
        x.insert(0,(labelsNames[i]))
        x = tuple(x)
        i = i + 1
        b.append(x)
    #print(b)
    return(b)
    #ipy_table.render()

In [72]:
def compara(codanaliseamostra1, codanaliseamostra2, av1, av2):
    codanaliseamostra1 = str(codanaliseamostra1)
    codanaliseamostra2 = str(codanaliseamostra2)
    

    cmd = ("SELECT am.observacao "
           "FROM analiseamostra am "
           "WHERE "
           "codanaliseamostra = " + codanaliseamostra1
           )
    res0 = query(cmd)
    #print(res0[0])
    ret = "<h2>" + res0[0] + "</h2>"
    
    cmd = ("SELECT am.codanaliseamostra, codamostra, hash, p.nome, "
           "(select count(distinct codtweet) from amostratweet where codamostra = am.codamostra) AS avaliar , "
           "(select count(distinct codtweet) from analisetweet where codanaliseamostra = am.codanaliseamostra) AS avaliados  "
           "FROM analiseamostra am "
           "INNER JOIN usuario p ON am.codusuario = p.codusuario "
           "WHERE "
           "codanaliseamostra = " + codanaliseamostra1 + 
           " OR " +
           "codanaliseamostra = " + codanaliseamostra2 +
           "ORDER BY codamostra, p.nome")
    res00 = query(cmd)
    res00.insert(0,('cod','amostra','hash','avaliador','avaliar','avaliados'))
    tbl00 = ipy_table.make_table(res00)
    #ipy_table.set_global_style(wrap=True)
    #ipy_table.apply_theme('basic')    
    
    # av 01
    cmd = ("SELECT cl.descricao "
           "FROM analisetweet at "
           "INNER JOIN classificacao cl ON cl.codclassificacao = at.codclassificacao "
           "WHERE codanaliseamostra = " + codanaliseamostra1 + " and cl.codclassificacao <> 36 "
           "ORDER BY cl.descricao"
          )
    res01 = query(cmd)
    topics = [x[0] for x in res01]
    count01 = Counter(topics)
    tbl01 = ipy_table.make_table(count01.most_common())
        
    # av 02
    cmd = ("SELECT cl.descricao "
           "FROM analisetweet at "
           "INNER JOIN classificacao cl ON cl.codclassificacao = at.codclassificacao "
           "WHERE codanaliseamostra = " + codanaliseamostra2 + " and cl.codclassificacao <> 36 "
           "ORDER BY cl.descricao"
          )
    res01 = query(cmd)
    topics = [x[0] for x in res01]
    count01 = Counter(topics)
    tbl02 = ipy_table.make_table(count01.most_common())
    
    ret = ret + tbl00._repr_html_()
    ret = ret + "<table><tr><td>" + av1 + ":" 
    ret = ret + tbl01._repr_html_() + '</td><td>' + av2 + ":"
    ret = ret + tbl02._repr_html_() + "</td></tr></table>"
    
    # carregando avaliação 01
    cmd = ("SELECT distinct on (at.codtweet) at.codtweet , at.codclassificacao "
           "FROM analisetweet at "
           "WHERE codanaliseamostra = " + codanaliseamostra1 + " "
           "AND codclassificacao in (233, 232) "
           "AND codtweet in (select codtweet from analisetweet "
           "where codanaliseamostra = " + codanaliseamostra2 + ") "
           "order by codtweet"
          )
    res = query(cmd)
    avMateus = [ x[1] for x in res ]

    # carregando avaliação 02
    cmd = ("SELECT distinct on (at.codtweet) at.codtweet , at.codclassificacao "
           "FROM analisetweet at "
           "WHERE codanaliseamostra = " + codanaliseamostra2 + " "
           "AND codclassificacao in (233, 232) "
           "and codtweet in (select codtweet from analisetweet "
           "where codanaliseamostra = " + codanaliseamostra1 + ") "
           "and at.codclassificacao <> 36  order by codtweet"
          )
    res = query(cmd)
    avEduarda = [ x[1] for x in res ]

    ret = ret + "<h2>Eixo 01: Relação com a notícia</h2>"
    #print("ed: ",len(avEduarda))
    #print(avEduarda)
    #print("mt: ",len(avMateus))
    #print(avMateus)

    ret = ret + ("<br>Kappa: " + av1 + " X " + av2 + ": %s" % cohen_kappa_score(avEduarda, avMateus))    
    tbl = ipy_table.make_table(matrixConfusao(avEduarda, avMateus, [232,233], ['relacionado','não relacionado']))
    ret = ret + tbl._repr_html_()
    
    # eixo 2
    # carregando avaliação mateus
    cmd = ("SELECT distinct on (at.codtweet) at.codtweet , at.codclassificacao "
           "FROM analisetweet at "
           "WHERE codanaliseamostra = " + codanaliseamostra1 + " "
           "AND codclassificacao in (234, 235, 236) "
           "AND codtweet in (select codtweet from analisetweet "
           "where codanaliseamostra = " + codanaliseamostra2 + " AND codclassificacao in (234, 235, 236)) "
           "order by codtweet"
          )
    res = query(cmd)
    avMateus = [ x[1] for x in res ]

    # carregando avaliação eduarda
    cmd = ("SELECT distinct on (at.codtweet) at.codtweet , at.codclassificacao "
           "FROM analisetweet at "
           "WHERE codanaliseamostra = " + codanaliseamostra2 + " "
           "AND codclassificacao in (234, 235, 236) "
           "and codtweet in (select codtweet from analisetweet "
           "where codanaliseamostra = " + codanaliseamostra1 + " AND codclassificacao in (234, 235, 236)) "
           "and at.codclassificacao <> 36  order by codtweet"
          )
    res = query(cmd)
    avEduarda = [ x[1] for x in res ]

    ret = ret + "<h2>Eixo 02: Fake News</h2>"
    ret = ret + ("<br>Kappa: " + av1 + " X " + av2 + ": %s" % cohen_kappa_score(avEduarda, avMateus))    
    #print("Eduarda e Mateus: %s" % cohen_kappa_score(avEduarda, avMateus))    
    
    tbl = ipy_table.make_table(matrixConfusao(avEduarda, avMateus, [234,235,236], ['Propagação','Contra','Réplica']))
    ret = ret + tbl._repr_html_()
    
    
    return HTML(ret)

# Resumo das avaliações dos tweets

## Notícia 01

In [73]:
compara(128, 127, "Eduarda", "Mateus")

TypeError: can only concatenate str (not "tuple") to str

## Notícia 03

In [56]:
compara(131, 145, "Eduarda", "Mateus")

## Notícia 4

In [57]:
compara(132, 146, "Eduarda", "Mateus")

## Notícia 05

In [58]:
compara(133, 144, "Eduarda", "Mateus")

## Notícia 06

In [59]:
compara(136, 149, "Eduarda", "Mateus")

## Notícia 07

In [60]:
compara(137, 150, "Eduarda", "Mateus")

## Notícia 08

In [61]:
compara(138, 147, "Eduarda", "Mateus")

## Notícia 09

In [62]:
compara(139, 151, "Eduarda", "Mateus")

## Notícia 10

In [63]:
compara(140, 152, "Eduarda", "Mateus")

## Notícia 14

In [64]:
compara(141, 153, "Eduarda", "Mateus")

## Notícia 15

In [65]:
compara(142, 154, "Eduarda", "Mateus")